# Example notebook

_Adapted from a notebook from Gdansk Summer School 'Picturing Quantum Weirdness 2023'_


In this problem sheet, we will get the hang of working with the basic generators of circuits and ZX-diagrams concretely, using matrix calculations. For this, we will use the `sympy` library.

In [14]:
from sympy import *
from sympy.physics.quantum import TensorProduct
from fractions import Fraction
from IPython.display import display

def T(*args):
    if len(args) == 1: return args[0]
    elif len(args) == 0: return Matrix([[1]])
    else: return TensorProduct(args[0], T(*args[1:]))

alpha = var("alpha")
beta = var("beta")
gamma = var("gamma")

With this library, we can construct matrices with `Matrix`. Then `*` is matrix multiplication, and `T` is tensor product.

Note `T` takes any number of arguments, e.g. $A \otimes B \otimes C$ is `T(A,B,C)`, and if you want to be fancy: $A^{\otimes n}$ is `T(*n*[A])`. (Python trivia: Why does that work??)

In [15]:
M = Matrix([[1,  2],
            [3,  4]])
display(M)
display(M * M * M)
display(T(M,M))

Matrix([
[1, 2],
[3, 4]])

Matrix([
[37,  54],
[81, 118]])

Matrix([
[1,  2,  2,  4],
[3,  4,  6,  8],
[3,  6,  4,  8],
[9, 12, 12, 16]])

We can also define some variables with `var` which can be used in mathmatical expressions, and substituted via `.subs(...)`. Variable names can be any string, but `sympy` knows how to pretty-print some variable names, e.g. Greek letters.

NOTE: $\sqrt{2}$ is `sqrt(2)`, $i$ is `I`, $\pi$ is `pi`, and $e^x$ is `exp(x)`, so phases $e^{i \alpha}$ are written `exp(i * alpha)`.

In [3]:
alpha = var("alpha")
phase = exp(I * alpha)
epi4 = exp(I * pi / 4)

display(alpha)
display(phase)
display(phase.subs(alpha, -pi / 2))
display(phase.subs(alpha, pi / 4) == epi4)
display(re(epi4) + I * im(epi4))

alpha

exp(I*alpha)

-I

True

sqrt(2)/2 + sqrt(2)*I/2

That should be enough to get going. If in doubt, [read the docs](https://docs.sympy.org/latest/index.html).

# Question 0

First some basics. Define matrices for:
 * `z0` := $|0\rangle$, `z1` := $|1\rangle$, `x0` := $|{+}\rangle$, `x1` := $|{-}\rangle$
 * `bz0` := $\langle 0|$, `bz1` := $\langle 1|$, `bx0` := $\langle {+}|$, `bx1` := $\langle {-}|$
 * `w` for the 2D identity matrix ("wire")
 * `swap` for the swap

Compute various products and tensor products and show the results are sensible.

In [4]:
#!# BEGIN SOLUTION
z0 = Matrix([[1],[0]])
z1 = Matrix([[0],[1]])
x0 = (1/sqrt(2)) * Matrix([[1], [1]])
x1 = (1/sqrt(2)) * Matrix([[1], [-1]])

bz0, bz1, bx0, bx1 = (m.adjoint() for m in (z0,z1,x0,x1))
w = eye(2)
swap = Matrix([[1, 0, 0, 0],
               [0, 0, 1, 0],
               [0, 1, 0, 0],
               [0, 0, 0, 1]])
#!# END SOLUTION

display(bz0 * z0)
display(bz0 * z1)
#!# BEGIN SOLUTION
display(T(bz0,bz0) + T(bz1,bz1))
display(swap * swap == T(w,w))
display(T(swap, w) * T(w, swap) * T(swap, w) == T(w, swap) * T(swap, w) * T(w, swap))
#!# END SOLUTION

Matrix([[1]])

Matrix([[0]])

Matrix([[1, 0, 0, 1]])

True

True

In [5]:
#!# BEGIN VISIBLE TESTS
if not 'weight' in locals():
    weight, name, description = [lambda x: lambda x: 0]*3
    
@weight(1)
@name("Question 0: z0")
@description("Test the z0 state")
def test_z0():
    assert student.z0 == z0

@weight(1)
@name("Question 0: z1")
@description("Test the z1 state")
def test_z1():
    assert student.z1 == z1

@weight(1)
@name("Question 0: x0")
@description("Test the x0 state")
def test_x0():
    assert student.x0 == x0

@weight(1)
@name("Question 0: x1")
@description("Test the x1 state")
def test_x1():
    assert student.x1 == x1

@weight(1)
@name("Question 0: bz0")
@description("Test the bz0 state")
def test_bz0():
    assert student.bz0 == bz0

@weight(1)
@name("Question 0: bz1")
@description("Test the bz1 state")
def test_bz1():
    assert student.bz1 == bz1

@weight(1)
@name("Question 0: bx0")
@description("Test the bx0 state")
def test_bx0():
    assert student.bx0 == bx0

@weight(1)
@name("Question 0: bx1")
@description("Test the bx1 state")
def test_bx1():
    assert student.bx1 == bx1
#!# END VISIBLE TESTS

#!# BEGIN HIDDEN TESTS
@weight(1)
@name("Question 0: wire")
@description("Test the 2D identity matrix")
def test_identity():
    assert student.w == w

@weight(1)
@name("Question 0: swap")
@description("Test the swap gate")
def test_swap():
    assert student.swap == swap
#!# END HIDDEN TESTS

# Question 1

Define a function that produces the matrix of a Z-spider. It should take 3 arguments: `m` for input legs, `n` for output legs, and `phase` for phase. The phase should have a default value of 0.

Build this function in (at least) 2 different ways:
 1. by building the $2^n \times 2^m$ matrix of the spider explicitly (call this function `zs`)
 2. by using sums, compositions, and tensor products of the generators from the previous question

Test your implementation by comparing the matrices for various choices of inputs, outputs, and phases. (Don't forget to check no-legged spiders!)

In [13]:
#!# BEGIN SOLUTION
def zs(m, n, phase=0):
    return Matrix([[
        (1 if i == 0 and j == 0 else 0) +
        (exp(I * phase) if i == 2**m - 1 and j == 2**n - 1 else 0)
      for i in range(2**m)] for j in range(2**n)])

def zs_gens(m, n, phase=0):
    return (
        T(*n*[z0]) * T(*m*[bz0]) +
        exp(I * phase) * T(*n*[z1]) * T(*m*[bz1])
    )
#!# END SOLUTION

display(all([[zs_gens(i, j, alpha) == zs(i, j, alpha) for i in range(4)] for j in range(4)]))

True

In [7]:
#!# BEGIN VISIBLE TESTS
import inspect
import re as regex

@weight(0)
@name("Question 1: zs method check")
@description("Checks if the zs function uses an explicit matrix construction")
def test_zs_method():
    source = inspect.getsource(student.zs)
    source = regex.sub(r"#.*", "", source) # Remove comments 
    
    assert "Matrix" in source
    assert "for" in source

@weight(0)
@name("Question 1: zs_gens method check")
@description("Checks if the zs function uses the generators and tensor products")
def test_zs_gens_method():
    source = inspect.getsource(student.zs_gens)
    source = regex.sub(r"#.*", "", source) # Remove comments 

    assert "T" in source
    assert "z0" in source or "z1" in source
#!# END VISIBLE TESTS

#!# BEGIN HIDDEN TESTS
import inspect
import re as regex

@weight(3)
@name("Question 1: zs")
@description("Test the z-spider generating function using an explicit matrix construction")
def test_zs():
    # Check method
    source = inspect.getsource(student.zs)
    source = regex.sub(r"#.*", "", source) # Remove comments 
    
    assert "Matrix" in source
    assert "for" in source
    
    # Check correctness
    for i in range(4):
        for j in range(4): 
            assert student.zs(i, j, alpha) == zs(i, j, alpha)

@weight(3)
@name("Question 1: zs_gens")
@description("Test the z-spider generating function using the generators")
def test_zs_gens():
    # Check method
    source = inspect.getsource(student.zs_gens)
    source = regex.sub(r"#.*", "", source) # Remove comments 

    assert "T" in source
    assert "z0" in source or "z1" in source

    # Check correcteness
    for i in range(4):
        for j in range(4): 
            assert student.zs_gens(i, j, alpha) == zs_gens(i, j, alpha)
#!# END HIDDEN TESTS